# Quantum Process Tomography Benchmarks

In [1]:
import sys
sys.path.insert(0, '../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from lindblad_channels import *
from quantum_tools import *
from experimental import *
from spam import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Recover Map

### Generate True Model, Full POVM and Inital

In [2]:
def generate_spam_benchmark(n=3, c1=1, c2=1):
    d = 2**n

    init_target = InitialState(d, c=c1)
    povm_target = POVM(d, c=c2)

    spam_target = SPAM(init = init_target,
                       povm = povm_target)
    
    return spam_target


def generate_spam_data(spam_target, N_spam=None, shots=1024):
    n = int(np.log2(spam_target.d))
    inputs_spam, _ = generate_pauliInput_circuits(n)
    N_spam = inputs_spam.shape[0]

    state = tf.repeat(spam_target.init.init[None,:,:], N_spam, axis=0)
    targets_spam = measurement(state, U_basis = inputs_spam, povm = spam_target.povm.povm)

    #add noise
    targets_spam = add_shot_noise(targets_spam, shots = shots)
    return inputs_spam, targets_spam


def generate_map_data(channel_target, spam_target, N_map=None, shots=1024, grid=True, batch_size=1000):
    n = int(np.log2(channel_target.d))
    inputs_map, _ = generate_pauli_circuits(n = n, 
                                            circuit_target=None,  
                                            N=N_map,
                                            grid=grid)
    U_prep, U_basis = inputs_map

    N_map = U_prep.shape[0]
    
    targets_map = []
    for batch_num in range(N_map//batch_size + 1):

        U_prep_batch = U_prep[batch_num*batch_size:(batch_num+1)*batch_size]
        U_basis_batch = U_basis[batch_num*batch_size:(batch_num+1)*batch_size]
        batch_size_current = U_prep_batch.shape[0]

        state = tf.repeat(tf.expand_dims(spam_target.init.init, axis=0), batch_size_current, axis=0)
        state = apply_unitary(state, U_prep_batch)
        state = channel_target.apply_channel(state)
        probs = measurement(state, U_basis_batch, spam_target.povm.povm)
        probs = add_shot_noise(probs, shots = shots)
        targets_map.append(probs)
    
    targets_map = tf.concat(targets_map, axis=0)

    return inputs_map, targets_map

## Three Qubits

### Generate Synthetic Data with Noise

In [8]:
n = 1
d = 2**n
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

spam_target1 = generate_spam_benchmark(n=n, c1=0.8, c2=0.8)
inputs_spam1, targets_spam1 = generate_spam_data(spam_target1, N_spam=None, shots=1024)

channel_target1 = DilutedKrausMap(U = generate_unitary(d), 
                                  c = 0.5,
                                  kraus_part = KrausMap(d=d, 
                                                        rank=d),
                                  )

#inputs_map1, targets_map1 = generate_map_data(channel_target1, spam_target1, N_map = 18, shots=20000)
inputs_map1, targets_map1 = generate_map_data(channel_target1, IdealSPAM(d), N_map = 18, shots=20000)

#saver(channel_target1, "data/three_qubit_channel_target.model")

### Fit SPAM

In [4]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

spam_model1 = SPAM(init = InitialState(d),
                  povm = CorruptionMatrix(d),
                  loss_function = ProbabilityMSE(),
                  optimizer = tf.optimizers.Adam(learning_rate=0.01)
                 )

spam_model1.pretrain(num_iter = 500, verbose=False)

spam_model1.train(inputs = inputs_spam1,
                  targets = targets_spam1,
                  num_iter = 1000,
                  verbose = False,
                 )

### Fit Quantum Channel

In [9]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

model_list1 = []
reps = 10
for i in tqdm(range(reps)):

    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                                rank = d**2,
                                                spam = IdealSPAM(d),
                                                ),
                            loss_function = ProbabilityMSE(),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            logger = Logger(loss_function=ProbabilityRValue(),
                                            loss_function_val = channel_fidelity_loss,
                                            sample_freq=10,
                                            N=500),
                        )

    model.train(inputs = inputs_map1,
                targets = targets_map1,
                inputs_val = None,
                targets_val = [channel_target1],
                num_iter = 2000,
                N = 500,
                verbose=True,
            )
    
    model = model_remove_gradients(model)
    
    model_list1.append(model)

  0%|          | 0/10 [00:00<?, ?it/s]

In [43]:
saver(model_list1, "data/three_qubit_benchmark.model")

## Four Qubits

### Generate synthetic data with noise

In [19]:
n = 4
d = 2**n
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

spam_target2 = generate_spam_benchmark(n=n, c1=0.8, c2=0.8)
inputs_spam2, targets_spam2 = generate_spam_data(spam_target2, N_spam=None, shots=1024)

channel_target2 = DilutedKrausMap(U = generate_unitary(d), 
                                  c = 0.5,
                                  kraus_part = KrausMap(d=d, 
                                                        rank=d),
                                  )

inputs_map2, targets_map2 = generate_map_data(channel_target2, spam_target2, N_map = 10000-6**n, shots=1024, batch_size=1000)

saver(channel_target2, "data/four_qubit_channel_target.model")

### Fit SPAM

In [20]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

spam_model2 = SPAM(init = InitialState(d),
                  povm = CorruptionMatrix(d),
                  loss_function = ProbabilityMSE(),
                  optimizer = tf.optimizers.Adam(learning_rate=0.01)
                 )

spam_model2.pretrain(num_iter = 500, verbose=False)

spam_model2.train(inputs = inputs_spam2,
                  targets = targets_spam2,
                  num_iter = 1000,
                  verbose = True,
                 )

  0%|          | 0/1000 [00:00<?, ?it/s]

step 0: loss = 0.0172
step 100: loss = 0.0009
step 200: loss = 0.0009
step 300: loss = 0.0009
step 400: loss = 0.0009
step 500: loss = 0.0009
step 600: loss = 0.0009
step 700: loss = 0.0009
step 800: loss = 0.0009
step 900: loss = 0.0009


### Fit Quantum Channel

In [26]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

model_list2 = []
reps = 1
for i in tqdm(range(reps)):
    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                                rank = d**2,
                                                spam = spam_model2,
                                                ),
                            loss_function = ProbabilityMSE(),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            logger = Logger(loss_function=ProbabilityRValue(),
                                            loss_function_val = channel_fidelity_loss,
                                            sample_freq=100,
                                            N = 500),
                        )

    model.train(inputs = inputs_map2,
                targets = targets_map2,
                inputs_val = None,
                targets_val = [channel_target2],
                num_iter = 4000,
                N = 500,
                verbose=True,
            )
    
    model = model_remove_gradients(model)
    
    model_list2.append(model)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

-0.012084975177130586 -0.04727783162063984
0.4902575924413417 -0.18147941692048447
0.8230102242516443 -0.3913869504277557
0.8422411892909888 -0.4535698006302818
0.8567639081958108 -0.5010340377239283
0.860300635410155 -0.5442657154100805
0.8626014463309566 -0.5827730978927838
0.8724187243911541 -0.6165278626844473
0.8682209524124747 -0.6452221836784513
0.8731909028653722 -0.6680559847016435
0.8763677361706633 -0.6881218490976555
0.8686706323780529 -0.7048756307835412
0.8753951441588387 -0.7181306844315237
0.8775671463548828 -0.7292910595823852
0.8771801412866731 -0.7390316375156958
0.8710221310193844 -0.7459789898447029
0.8767348667364228 -0.7521851074323261
0.8750464274806875 -0.7576123734884442
0.8731433230023202 -0.7619315940378866
0.8818657757450837 -0.7656652762417911
0.8748051349322205 -0.7689665609279888
0.880745898245745 -0.7719745166546483
0.8832597209368509 -0.773738890485677
0.8746537860547113 -0.7758206275144583
0.8767237235173728 -0.7780146017654467
0.8805556288388069 -0.7

In [27]:
saver(model_list2, "data/four_qubit_benchmark.model")